<a href="https://colab.research.google.com/github/Ryan-R-Davis/Baseball-Steals/blob/main/Scrape_RE_Matrixes_from_BP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
!pip install MechanicalSoup
import mechanicalsoup
import time
from google.colab import drive

drive.mount('/content/drive')
path = '/content/drive/My Drive/Portfolio/Baseball Steals/Clean Data.csv'



In [ ]:
years = range(1950, 2020)
url = "https://legacy.baseballprospectus.com/sortable/index.php?cid=2800999"
browser = mechanicalsoup.Browser()
page = browser.get(url)
if page.status_code == 200:
  print("Page loaded sucessfully")
else:
  print("Page failed to load")

In [ ]:
cleanData = []
cleanRow = []
cols = ["Year", "BaseOuts", "ExpectedRuns"]

for year in years:
  #print(year)
  time.sleep(.5) # small delay in order to keep from overloading the server
  url = "https://legacy.baseballprospectus.com/sortable/index.php?cid=2800999"
  browser = mechanicalsoup.Browser()
  page = browser.get(url)
  form = page.soup.select("form")[2]
  form.select("option")[0]["value"] = year
  page = browser.submit(form, page.url)
  Tables = page.soup.find_all("table")
  for i in range(9,63,7): # Selects desired columns from one/each row of the raw table
    row = Tables[4].find_all("td")[i:i+5]
    for ii in range(0,3): # for each Base+Outs situation creates an individual row
      cleanRow.append(row[0].text) # starts row with the year
      cleanRow.append(row[1].text + f"Outs{ii}") # combines the base and outs into one variable, BaseOuts. adds it to the row
      cleanRow.append(float(row[ii+2].text)) # adds the numeric value for the expected runs in that situation to the row. 
      #print(cleanRow)
      cleanData.append(cleanRow)
      cleanRow = []

df = pd.DataFrame(cleanData, columns=cols)
df.set_index('Year', inplace=True)
print(df)

In [4]:
with open(path, 'w', encoding = 'utf-8-sig') as f:
  df.to_csv(f)